In [ ]:
import pickle

import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

## НАСТРОЙКА

In [ ]:
TARGET_NAME = #колонка с таргетом в датасете
IMAGES_DIR = #папка с картинками
DATASET_PATH = #путь к датасету полный
FILENAME_COL = #колонка с именем файла в датасете
TARGET_SIZE = (224, 224) #tuple, размер ресайзинга
BATCH_SIZE = 32 #размер батча
EPOCHS = 10 #кол-во эпох
STEPS_PER_EPOCH = #шаги за эпоху
VALIDATION_STEPS = #шаги валидации
INUT_SHAPE = (TARGET_SIZE[0], TARGET_SIZE[1] ,3)#размер входа для резнет
LEARNING_RATE=0.0001

In [ ]:
optimizer = Adam(learning_rate=LEARNING_RATE)


def load_train(path):
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       validation_split=0.25,
                                       horizontal_flip=True,
                                       rotation_range=30,)
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=pd.read_csv(DATASET_PATH),
        directory=IMAGES_DIR,
        x_col=TARGET_NAME,
        y_col=IMAGES_PATH,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training',
        seed=12345)

    return train_gen_flow

def load_test(path):
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=pd.read_csv(DATASET_PATH),
        directory=IMAGES_DIR,
        x_col=TARGET_NAME,
        y_col=IMAGES_PATH,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation',
        seed=12345)

    return test_gen_flow

def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape,
                        weights='imagenet',
                        include_top=False)
    
    backbone.trainable = False

    model = Sequential()
    model.add(backbone)
    #новая голова
    model.add(GlobalAveragePooling2D())
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1, activation='softmax'))
    model.compile(loss=BinaryCrossentropy(), optimizer=optimizer,
                  metrics=['accuracy'])

    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=10,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, epochs=epochs, shuffle=True)

    return model


In [ ]:
model = train_model(
    create_model(INUT_SHAPE),
    load_train(),
    load_test(),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS
    )

In [ ]:
pickle.dump(model, open('baseline.pkl', 'wb'))

In [ ]:
loss_train_curve = model.history["loss"]
loss_val_curve = model.history["val_loss"]
plt.plot(loss_train_curve, label = "Train")
plt.plot(loss_val_curve, label = "Validation")
plt.legend(loc = 'upper right')
plt.title("Loss")
plt.show()

In [ ]:
acc_train_curve = model.history["accuracy"]
acc_val_curve = model.history["val_accuracy"]
plt.plot(acc_train_curve, label = "Train")
plt.plot(acc_val_curve, label = "Validation")
plt.legend(loc = 'lower right')
plt.title("Accuracy")
plt.show()